In [ ]:
import re
import spacy
from spacy.matcher import Matcher
import fitz

# ... (Descargar el modelo de spaCy como lo tenías antes)


def extraer_info(ruta_pdf):  # Corregido: Recibir la ruta, no el texto
    metadatos = {}  # Cambiado el nombre a metadatos para mayor claridad
    
    try:   
        with fitz.open(ruta_pdf) as doc:
            texto_completo = ""
            for pagina in doc:
                texto_completo += pagina.get_text()

            texto_completo = preprocesar_texto(texto_completo)

            # ... (Código para extraer nombre, ISSN, fecha, entidad, año, rama igual que antes)

            # Directores (usando spaCy - Versión ligeramente simplificada)
            metadatos["directores"] = {"senado": {}, "camara": {}}
            # ... (Código para encontrar el texto_directores igual que antes)


            if texto_directores: # Verificar si texto_directores no está vacío
                doc_directores = nlp(texto_directores)

                for ent in doc_directores.ents:
                    if ent.label_ == "PER":
                        nombre = ent.text.strip()
                        cargo = None # Inicializar cargo dentro del bucle para cada entidad
                        # Buscar cargo en la misma frase o en las frases siguientes
                        for sent in doc_directores.sents: #sentencias
                            if nombre in sent.text: #en las sentencias, se valida si el nombre existe
                                for token in sent: #se itera en las sentencias
                                    if "SECRETARIO" in token.text.upper() and "GENERAL" in token.text.upper():
                                        cargo = token.text
                                        if "SENADO" in cargo.upper():
                                            metadatos["directores"]["senado"] = {"nombre": nombre, "cargo": cargo}
                                        elif "CÁMARA" in cargo.upper():
                                            metadatos["directores"]["camara"] = {"nombre": nombre, "cargo": cargo}
                                        break # Romper el bucle si se encuentra el cargo
                
                                #si no se ha encontrado el cargo en la frase actual, mirar la siguiente.    
                                if cargo:
                                    break


            # Documentos (combinando regex y spaCy)
            documentos = []
            pattern_tipo_documento = r"(?i)(PONENCIA|ACTA|PROYECTO DE LEY|INFORME|RESOLUCIÓN|CONCEPTO|PROPOSICIÓN|CONSTANCIA|OBJECIONES|CONCEPTOS JURÍDICOS|LEYES SANCIONADAS|PRESENTACIÓN)(.*)"

            for match in re.finditer(pattern_tipo_documento, texto_completo):
                tipo_documento = match.group(1).strip()
                texto_restante = match.group(2)

                doc_spacy = nlp(texto_restante)

                subtitulo = ""
                subsubtitulo = ""

                # Extraer subtítulo (primera línea no vacía)
                for sent in doc_spacy.sents:
                    subtitulo = sent.text.strip()
                    if subtitulo:
                        break

                # Extraer subsubtítulo (adaptar según la estructura de tus PDFs)
                if subtitulo:
                    texto_despues_subtitulo = texto_restante[texto_restante.find(subtitulo) + len(subtitulo):].strip()

                    # Ejemplo: Buscar entre comillas dobles
                    match_subsubtitulo = re.search(r'"(.*?)"', texto_despues_subtitulo)
                    if match_subsubtitulo:
                        subsubtitulo = match_subsubtitulo.group(1)

                    else: #si no se encuentra entre comillas, buscar la siguiente frase
                         for sent in nlp(texto_despues_subtitulo).sents:
                            posible_subsubtitulo = sent.text.strip()
                            if posible_subsubtitulo:
                                subsubtitulo = posible_subsubtitulo
                                break




                documentos.append({
                    "tipo_documento": tipo_documento,
                    "subtitulo": subtitulo,
                    "subsubtitulo": subsubtitulo
                })

            metadatos["documentos"] = documentos # Agregar la lista de documentos a metadatos


    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo PDF: {ruta_pdf}")
        return None
    except Exception as e:
        print(f"Error al procesar el PDF {ruta_pdf}: {e}")
        return None

    return metadatos


# ... (resto de las funciones auxiliares: extraer_con_regex, preprocesar_texto, romano_a_entero)


# --- Ejemplo de Uso ---
ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328 XXV 110_64.pdf"  # Reemplaza con la ruta correcta
datos_extraidos = extraer_info(ruta_gaceta) # Pasar la ruta directamente

if datos_extraidos:
    print(datos_extraidos)